# Managing SoS Projects

## Tracked and Untracked files

The most important concept for managing SoS projects is <font color='red'>tracked files</font>, which are all input, output, and dependent files of steps of an executed workflow. The files are tracked by SoS using a signature system that records the name and content of the files. For a file to be tracked,

* the file has to be included in `input`, `output`, or `depends` statements of one of the steps
* the workflow has to be executed with [signature](Execution_of_Workflow.html) (namely, not with `-s ignore`, which is the default mode under Jupyter notebook).

Let us create a SoS script that generate some tracked and untracked files

In [1]:
%sandbox --dir temp
%set -s default -v2

parameter: name='tracked_f1'
[0]
output:  name
sh:
    dd if=/dev/urandom of=${name} count=1000

[1]
output:  'd1/tracked_f2'
sh:
    dd if=/dev/urandom of=${output} count=5000
    dd if=/dev/urandom of=d1/untracked_f4 count=50

[2]
output:  'd2/d3/tracked_f3'
sh:
    dd if=/dev/urandom of=${output} count=600

sos options is set to "-s default -v2"


INFO: Running default_0: 
1000+0 records in
1000+0 records out
512000 bytes transferred in 0.046287 secs (11061407 bytes/sec)
INFO: Running default_1: 
5000+0 records in
5000+0 records out
2560000 bytes transferred in 0.174087 secs (14705287 bytes/sec)
50+0 records in
50+0 records out
25600 bytes transferred in 0.001732 secs (14779653 bytes/sec)
INFO: Running default_2: 
600+0 records in
600+0 records out
307200 bytes transferred in 0.023974 secs (12813913 bytes/sec)
INFO: Workflow default (ID=5cad13496bca14a2) is executed successfully.


## -- Preview output --

input: d1/tracked_f2
output: d2/d3/tracked_f3

> d2/d3/tracked_f3 (300.0 KiB):

This workflow creates three tracked files `tracked_f1`, `d1/tracked_f2`, `d2/d3/tracked_f3`, and as a side effect creates a `d1/untracked_f4` file.

## Remove tracked or untracked files

Subcommand `remove` is used to remove untracked files and directories to keep the project directory clean. It can also be used to remove some files so that it can be re-generated. The latter is needed because SoS would not generate removed intermediate files as long as they are not actually used in a another step.

When we re-run the workflow, all steps are ignored.

In [2]:
%sandbox --dir temp
%rerun -v2

INFO: Running default_0: 
INFO: Step default_0 (index=0) is ignored due to saved signature
INFO: Running default_1: 
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: 
INFO: Step default_2 (index=0) is ignored due to saved signature
INFO: Workflow default (ID=5cad13496bca14a2) is executed successfully.


## -- Preview output --

input: d1/tracked_f2
output: d2/d3/tracked_f3

> d2/d3/tracked_f3 (300.0 KiB):

If we remove an intermediate file, the workflow would not complain during re-execution

In [3]:
%sandbox --dir temp
!rm -f d1/tracked_f2
%rerun -v2

INFO: Running default_0: 
INFO: Step default_0 (index=0) is ignored due to saved signature
INFO: Running default_1: 
INFO: Step default_1 (index=0) is ignored due to saved signature
INFO: Running default_2: 
INFO: Step default_2 (index=0) is ignored due to saved signature
INFO: Workflow default (ID=5cad13496bca14a2) is executed successfully.


## -- Preview output --

input: d1/tracked_f2
output: d2/d3/tracked_f3

> d2/d3/tracked_f3 (300.0 KiB):

You can use option `-s force` to re-generate all files, but it is easier to just remove the intermediate file and its signature using command `remove`. Option `-y` stands for `-yes` and you would be prompted each time for file removal without this option.

In [4]:
%sandbox --dir temp
!sos remove -y d1/tracked_f2
%rerun -v2

INFO: 3 tracked files from 1 run are identified.


Remove tracked file d1/tracked_f2 and its signature


INFO: Running default_0: 
INFO: Step default_0 (index=0) is ignored due to saved signature
INFO: Running default_1: 
5000+0 records in
5000+0 records out
2560000 bytes transferred in 0.167291 secs (15302683 bytes/sec)
50+0 records in
50+0 records out
25600 bytes transferred in 0.001761 secs (14537528 bytes/sec)
INFO: Running default_2: 
600+0 records in
600+0 records out
307200 bytes transferred in 0.020721 secs (14825398 bytes/sec)
INFO: Workflow default (ID=5cad13496bca14a2) is executed successfully.


## -- Preview output --

input: d1/tracked_f2
output: d2/d3/tracked_f3

> d2/d3/tracked_f3 (300.0 KiB):

You can use the `sos remove` command to remove all untracked files with option `-u` (untracked).

In [5]:
%sandbox --dir temp
!ls -R

__interactive__.sos
d1
d2
result.txt
size.txt
tracked_f1

./d1:
tracked_f2
untracked_f4

./d2:
d3

./d2/d3:
tracked_f3


In [6]:
%sandbox --dir temp
!sos remove . -u -y

INFO: 3 tracked files from 1 run are identified.


Remove untracked file ./d1/untracked_f4


or remove all tracked files with option `-t` (tracked)

In [7]:
%sandbox --dir temp
!sos remove . -t -y

INFO: 3 tracked files from 1 run are identified.


Remove tracked file ./d1/tracked_f2 and its signature
Remove tracked file ./d2/d3/tracked_f3 and its signature
Remove ./d2/d3 with tracked files
Remove ./d1 with tracked files
Remove ./d2 with tracked files


In [8]:
%sandbox --dir temp
!ls -R

__interactive__.sos
result.txt
size.txt
tracked_f1


Because files under the currently directly are often important, they are by default not removed with these global options (`-t` or `-u`). You will have to remove them explicitly if so desired.

In [9]:
%sandbox --dir temp
!sos remove tracked_f1 -y

INFO: 3 tracked files from 1 run are identified.


Remove tracked file tracked_f1 and its signature


## Archving a project

Let us re-run the project and create all files

In [10]:
%sandbox --dir temp
%rerun

INFO: Running default_0: 
1000+0 records in
1000+0 records out
512000 bytes transferred in 0.033955 secs (15078738 bytes/sec)
INFO: Running default_1: 
5000+0 records in
5000+0 records out
2560000 bytes transferred in 0.168573 secs (15186308 bytes/sec)
50+0 records in
50+0 records out
25600 bytes transferred in 0.001733 secs (14773553 bytes/sec)
INFO: Running default_2: 
600+0 records in
600+0 records out
307200 bytes transferred in 0.021039 secs (14601448 bytes/sec)
INFO: Workflow default (ID=5cad13496bca14a2) is executed successfully.


## -- Preview output --

input: d1/tracked_f2
output: d2/d3/tracked_f3

> d2/d3/tracked_f3 (300.0 KiB):

For archiving and reproducibility purposes, it is often needed to create an archive for the analysis so that you can refer to it later. This is very easy to do with the `sos pack` command.

To use this feature, you first need to get the workflow session that you would like to pack, using the ID that is printed at the end of the execution. You do not have to specify the complete IDs, only the first few characters (even no character) as long as it can be used to identify an unique session. Because we have only executed one workflow, we can run

In [11]:
%sandbox --dir temp
!sos pack -o myproj.sar -y

INFO: Checking tracked_f1
INFO: Checking d1/tracked_f2
INFO: Checking d2/d3/tracked_f3
INFO: Archiving 3 files (3.2 MiB)...
INFO: Adding __interactive__.sos
INFO: Adding tracked_f1
INFO: Adding d1/tracked_f2
INFO: Adding d2/d3/tracked_f3
INFO: Adding runtime files


You can use options `--include` and `--exclude` to include or exclude specific files or directories to the archive. For example, if you would like to archive the untracked file `d1/untracked_f4`, you can add this file, or the whole directory using command

In [12]:
%sandbox --dir temp
!sos pack -i d1 -o myproj_all.sar -y

INFO: Checking d2/d3/tracked_f3
INFO: Checking d1/untracked_f4
INFO: Checking tracked_f1
INFO: Checking d1/tracked_f2
INFO: Archiving 4 files (3.2 MiB)...
INFO: Adding __interactive__.sos
INFO: Adding d2/d3/tracked_f3
INFO: Adding d1/untracked_f4
INFO: Adding tracked_f1
INFO: Adding d1/tracked_f2
INFO: Adding runtime files


## Unpacking an archive

Now that we have an archive of the project, we can remove all files under the directory

In [13]:
%sandbox --dir temp
!rm -rf .sos d1 d2 tracked_f1 *.dot *.sos

In [14]:
%sandbox --dir temp
!ls

myproj.sar
myproj_all.sar
result.txt
size.txt


We can unpack the archive using command `sos unpack`

In [15]:
%sandbox --dir temp
!sos unpack myproj.sar -y

INFO: Extracting tracked_f1
INFO: Extracting d1/tracked_f2
INFO: Extracting d2/d3/tracked_f3


In [16]:
%sandbox --dir temp
!ls

d1
d2
myproj.sar
myproj_all.sar
result.txt
size.txt
tracked_f1


As you have noticed, because the script is specified in Jupyter notebook, it is archived in name `__interactive__.sos`, but it is not unpacked. This is because sos script does not have to be in the current directory and it can be dangerous to overwrite a local script with an archived one. To unpack the script, use option `-s`

In [17]:
%sandbox --dir temp
!sos unpack myproj.sar -y -s

INFO: Extracting __interactive__.sos
INFO: Ignore identical tracked_f1
INFO: Ignore identical d1/tracked_f2
INFO: Ignore identical d2/d3/tracked_f3


and now you can check the script that is used to generate this archive

In [18]:
%sandbox --dir temp
!cat __interactive__.sos

parameter: name='tracked_f1'
[0]
output:  name
sh:
    dd if=/dev/urandom of=${name} count=1000

[1]
output:  'd1/tracked_f2'
sh:
    dd if=/dev/urandom of=${output} count=5000
    dd if=/dev/urandom of=d1/untracked_f4 count=50

[2]
output:  'd2/d3/tracked_f3'
sh:
    dd if=/dev/urandom of=${output} count=600